In [ ]:
import os

def allsymbols(path):
    import MetaTrader5 as mt5
    if not mt5.initialize(path):
        print("initialize() failed, error code =",mt5.last_error())
        quit()

    allsymbol=[]
    symbols=mt5.symbols_get()
    count=0
    for s in symbols:
        count+=1
        allsymbol.append(s.name)
    return allsymbol

def readdata(market,tf,barre,path):
    from datetime import datetime
    import MetaTrader5 as mt5
    import pandas as pd

    if not mt5.initialize(path):
        print("initialize() failed, error code =",mt5.last_error())
        quit()
    
    mk=str(market)
    TF="mt5.TIMEFRAME_"+str(tf)
    bar=barre
     
    while True:

        try:
            rates = mt5.copy_rates_from_pos(mk, eval(TF), 0, bar) #1700
            if len(rates)>0:
                break
        except:
            continue

    rates_frame = pd.DataFrame(rates)
    rates_frame['time']=pd.to_datetime(rates_frame['time'], unit='s')
    rates_frame=rates_frame[["time","open","high","low","close","tick_volume"]]
    rates_frame.columns=(["time","open","high","low","close","volume"])
    rates_frame=rates_frame.set_index(["time"])
    rates_frame=rates_frame[(rates_frame.T != 0).any()]
    return rates_frame

def get_poin_values(path,simb):

    import MetaTrader5 as mt5
    if not mt5.initialize(path):
        print("initialize() failed, error code =",mt5.last_error())
        quit()
    symbol_info=mt5.symbol_info(simb)
    if symbol_info!=None:
        symbol_info_dict = mt5.symbol_info(simb)._asdict()
    mt5.shutdown()    
    tmp=symbol_info_dict['trade_tick_value']/symbol_info_dict['point']
    return tmp

def readdata_MT5_Date(market,tf,path,start,stop):
    from datetime import datetime
    import MetaTrader5 as mt5
    import pandas as pd
    
    while True:

        if not mt5.initialize(path):
            print("initialize() failed, error code =",mt5.last_error())
            quit()
        mk=str(market)
        TF="mt5.TIMEFRAME_"+str(tf)

        start_y , start_m , start_d = start.split("-")
        stop_y , stop_m , stop_d = stop.split("-")
        
        utc_from = datetime(int(start_y), int(start_m), int(start_d))
        utc_to = datetime(int(stop_y), int(stop_m), int(stop_d))
        
        rates = mt5.copy_rates_range(mk, eval(TF), utc_from, utc_to) 
    

        rates_frame = pd.DataFrame(rates)
        rates_frame['time']=pd.to_datetime(rates_frame['time'], unit='s')
        rates_frame=rates_frame[["time","open","high","low","close","tick_volume"]]
        rates_frame.columns=(["time","open","high","low","close","volume"])
        rates_frame=rates_frame.set_index(["time"])
        #rates_frame=rates_frame[(rates_frame.T != 0).any()]
        if rates_frame is not None:
            return rates_frame
            break

In [ ]:
def rnd_gene():
    """
    Funzione che attiva un gene casuale
    cioè un indice di colonna nella tabella rules.
    """
    return np.random.randint(0, len(rules.columns) - 1)


def rnd_dna():
    """
    Funzione che genera un dna casuale (a singolo cromosoma, solo regole in "and" logico).
    """
    dna = []
    for i in range(0, DNA_SIZE):
        dna.append(rnd_gene())
    return dna


def first_generation():
    """
    Funzione che genera un dataframe con la popolazione iniziale casuale. 
    Ogni riga corrisponde a un DNA cioè ad un investitore artificiale.
    """
    population = pd.DataFrame(index = range(POPULATION_SIZE), columns = range(DNA_SIZE))
    for i in range(0, POPULATION_SIZE):
        population.iloc[i] = rnd_dna()
    return population


def dna_formula(dna):
    """
    Funzione che calcola la formula di un DNA (a singolo cromosoma).
    """
    formula = ""
    for gene in dna:
        if len(formula) != 0:
            formula += " & " ## "and"
        formula += "("+rule_formulas[gene]+")"
    return formula


@numba.jit(nopython = True) # ACCELARAZIONE NUMBA: aggiungo un'annotazione per compilare il metodo setups() con Numba
def setups(dna):
    """
    Funzione che calcola i setups di un dna (ad unico cromosoma) su tutte le regole
    Versione che sostituisce la precedente funzione in Pandas
    """
    setups = None
    for gene in dna:
        if setups is None:
            setups = rules_array[gene] # ACCELARAZIONE NUMPY: usiamo l'array al posto del dataframe
        else:
            setups = setups & rules_array[gene] # ACCELARAZIONE NUMPY: usiamo l'array al posto del dataframe
    return setups


def plot_equity(dna):
    """
    Funzione che grafichi l'equity line risultatnte 
    dato un DNA.
    """
    gains = all_gains(dna)
    times, gains = zip(*gains)
    profits = np.array(gains).cumsum()
    equity = pd.Series(profits, times)
    equity.plot(figsize=(14, 8), grid = True)

    
def sort_population(population):
    """
    Funzione per operare un ordinamento della popolazione
    rispetto alla fitness function
    """
    scores = population.apply(lambda dna: fitness(dna), axis = 1)
    sorted_population = population.copy()
    sorted_population["Score"] = scores
    sorted_population.sort_values(by = ["Score"], ascending = [False], inplace = True)
    sorted_population.reset_index(drop = True, inplace = True)
    sorted_population.drop(sorted_population.columns[[-1,]], axis = 1, inplace = True)
    return sorted_population


def mutation(mut_prob,dna1):
    """
    Funzione che muta un DNA cambiando ogni gene con un gene casuale
    con probabilità MUTATION_PROB.
    """
    dna = []
    for i in range(DNA_SIZE):
        if np.random.random() < mut_prob:
            dna.append(rnd_gene())
        else:
            dna.append(dna1[i])
    return dna


def crossover(dna1, dna2):
    """
    Funzione che, presi due DNA, in modo randomico, 
    prende il gene di uno dei due genitori
    """
    dna = []
    for i in range(DNA_SIZE):
        if np.random.randint(2) == 0:
            dna.append(dna1[i])
        else:
            dna.append(dna2[i])
    return dna

In [ ]:
def stack_sort(gains,secondary_equity):
    c=np.vstack((gains,secondary_equity))  
    d=sorted(c,key=lambda x: x[0])
    return d

def STD(dna):
    gains = all_gains(dna)
    ops = len([gain for (time, gain) in gains])
    if (ops >= MIN_OPERATIONS)&(sum([gain for (time, gain) in gains])>0):  
        se_tmp = secondary_OOS(secondary_equity,OOS_date)
        gains=stack_sort(gains,se_tmp)  
        times, gains = zip(*gains)
        return np.std(np.cumsum(gains))
    else:
        return -9999999999999999999

def profit(dna):
    """
    Fitness Function basata sul Profit totale.
    """
    gains = all_gains(dna)
    ops = len([gain for (time, gain) in gains])
    if ops >= MIN_OPERATIONS:  
        se_tmp = secondary_OOS(secondary_equity,OOS_date)
        gains=stack_sort(gains,se_tmp)   
        return round(sum([gain for (time, gain) in gains]),2)
    else:
        return 0
    
def kestner(dna):
    from scipy import stats    
    """
    Fitness Function basata sul Kestner Ratio.
    """
    gains = all_gains(dna)
    ops = len([gain for (time, gain) in gains])
    if ops >= MIN_OPERATIONS:  
        se_tmp = secondary_OOS(secondary_equity,OOS_date)
        gains=stack_sort(gains,se_tmp)  
        times, gains = zip(*gains)
                
        index = np.array(np.arange(1,len(times) + 1))

        x = index
        y = np.cumsum(gains)
        gradient, intercept, r_value, p_value, std_err = stats.linregress(x,y)

        if std_err != 0 and len(index) > 0:
            return round(gradient / (std_err * len(index)),5)
        else:
            return -999999
    else:
        return -999999    
    
def max_drawdown(dna):
    """
    Fitness Function basata sul Draw Down.
    """
    gains = all_gains(dna)
    ops = len([gain for (time, gain) in gains])
    if ops >= MIN_OPERATIONS:
        se_tmp = secondary_OOS(secondary_equity,OOS_date)
        gains=stack_sort(gains,se_tmp)   
        eq = np.cumsum([i[1] for i in gains])

        eq_top_list = []
        eq_top = 0
        for el in eq:
            if el > eq_top:
                eq_top = el
            eq_top_list.append(eq_top)

        #print(len(eq),len(eq_top_list))

        dd = eq - eq_top_list

        if ops >= MIN_OPERATIONS and len(dd) != 0:

            return min(dd)
        else:
            return -np.inf
    else:
        return -np.inf
    
    
def avg_trade(dna):
    """
    Fitness Function basata sul AvgTrade.
    """
    gains = all_gains(dna)
    ops = len([gain for (time, gain) in gains])
    if ops >= MIN_OPERATIONS:
        se_tmp = secondary_OOS(secondary_equity,OOS_date)
        gains=stack_sort(gains,se_tmp)   
        my_sum = sum([gain for (time, gain) in gains])

        if ops == 0 or ops < MIN_OPERATIONS:
            return 0
        else:
            return my_sum / ops
    else:
        return 0
    
    
def gross_profit(dna):
    """
    Fitness Function basata sul Gross Profit.
    """
    gains = all_gains(dna)
    ops = len([gain for (time, gain) in gains])
    if ops >= MIN_OPERATIONS:
        se_tmp = secondary_OOS(secondary_equity,OOS_date)
        gains=stack_sort(gains,se_tmp)   
        pos_sum = sum([gain for (time, gain) in gains if gain > 0])

        if ops < MIN_OPERATIONS:
            return 1
        else:
            return pos_sum
    else:
        return 1
    
def profit_factor(dna):
    """
    Fitness Function basata sul Profit Factor.
    """
    gains = all_gains(dna)
    ops = len([gain for (time, gain) in gains])
    if ops >= MIN_OPERATIONS:
        se_tmp = secondary_OOS(secondary_equity,OOS_date)
        gains=stack_sort(gains,se_tmp)   
        pos_sum = sum([gain for (time, gain) in gains if gain > 0])
        neg_sum = sum([gain for (time, gain) in gains if gain < 0])
        #print(pos_sum,neg_sum)

        if neg_sum == 0 or ops < MIN_OPERATIONS:
            return 0
        else:
            return round(abs(pos_sum / neg_sum),2)
    else:
        return 0
    
def SQN(dna):
    """
    Fitness Function basata sul SQN.
    """
    gains = all_gains(dna)
    ops = len([gain for (time, gain) in gains])    
    if ops >= MIN_OPERATIONS:
        se_tmp = secondary_OOS(secondary_equity,OOS_date)
        gains=stack_sort(gains,se_tmp)
        my_sum = np.sum([i[1] for i in gains])
        my_std = np.std([gain for (time, gain) in gains])
        sqn = np.sqrt(len(gains))*(my_sum/len(gains))/ my_std  
        return sqn
    else:
        return 0
    
def fitness(dna):
    """
    Funzione che associa la funzione di fitness
    su cui operare l'ordinamento al termine di ciascuna generazione
    """
    if FITNESS_FUNCTION == "Profit":
        return profit(dna)
    if FITNESS_FUNCTION == "AvgTrade":
        return avg_trade(dna)
    if FITNESS_FUNCTION == "ProfitFactor":
        return profit_factor(dna)
    if FITNESS_FUNCTION == "GrossProfit":
        return gross_profit(dna)
    if FITNESS_FUNCTION == "Profit/MaxDrawDown":
        return profit(dna) / -max_drawdown(dna)
    if FITNESS_FUNCTION == "AvgTrade/MaxDrawDown":
        return avg_trade(dna) / -max_drawdown(dna)
    if FITNESS_FUNCTION == "KestnerRatio":
        return kestner(dna)
    if FITNESS_FUNCTION == "StandardDeviation":
        return STD(dna)
    if FITNESS_FUNCTION == "SQN":
        return SQN(dna)

In [ ]:
def next_generation(population):
    """
    Funzione che riporta la generazione successiva
    """
    n = len(population.index)
    n_best = int(n * BEST_DNA_RATIO)
    n_cross = int(n * CROSS_DNA_RATIO)
    n_mutate = n - n_best - n_cross

    new_population = pd.DataFrame(index = range(POPULATION_SIZE), columns = range(DNA_SIZE))

    # copia i migliori DNA
    for i in range(n_best):
        new_population.iloc[i] = population.iloc[i]

    # aggiunge gli incroci
    for i in range(n_cross):
        dna1 = population.iloc[np.random.randint(n_best)]
        dna2 = population.iloc[np.random.randint(n_best)]
        dna = crossover(dna1, dna2)
        new_population.iloc[n_best + i] = dna

    # aggiunge le mutazioni
    for i in range(n_mutate):
        dna1 = population.iloc[np.random.randint(n_best)]
        dna = mutation(MUTATION_PROB, dna1)
        new_population.iloc[n_best + n_cross + i] = dna

    return new_population


def print_best(population, gen_num):
    """
    Funzione per la stampa delle performance del migliore investitore/DNA
    di una determinata generazione.
    """
    best_dna = population.iloc[0]    
    #tf = total_profit(best_dna)
    global tf
    tf = fitness(best_dna)
    #print("generation", gen_num, ":", round(tf, 2), " ", dna_formula(best_dna))
    return "generation", gen_num, ":", round(tf, 2), " ", dna_formula(best_dna) 

from IPython.display import clear_output
def evolution():
    """
    Funzione che definisce il ciclo principale dell'algoritmo genetico
    """
    best_dnas = []
    global log
    log = []
    population = first_generation()
    population = sort_population(population)
    best_dnas.append(population.iloc[0].values)
    
    
    # evolve for NUM_GENERATIONS generations
    for i in range(1, NUM_GENERATIONS):
        population = next_generation(population)
        population = sort_population(population)
        best_dnas.append(population.iloc[0].values)
        
        #print_best(population, i)
        clear_output(wait=True)
        print(print_best(population, i))
        log.append(tf)
        plot_equity_GSA(population.iloc[0])
        plt.legend()
        plot_equity_secondary(secondary_equity)
        plt.legend()
        plt.show()
        plot_equity_full(population.iloc[0],secondary_equity)
        plt.title("Aggregate Strategies")
        plt.show()
    return best_dnas

In [ ]:
def plot_equity(dna):
    """
    Funzione che grafichi l'equity line risultatnte 
    dato un DNA.
    """
    gains = all_gains_full(dna)
    if len(gains)>0:
        times, gains = zip(*gains)
        profits = np.array(gains).cumsum()
        equity = pd.Series(profits, times)
        equity.plot(figsize=(14, 8), grid = True, label="main strategies")
        plt.axvline(x=OOS_date ,color="red")
def plot_equity_secondary(secondary_equity):
    """
    Funzione che grafichi l'equity line risultatnte 
    dato un DNA.
    """
    if len(secondary_equity)>2:
        gains = secondary_equity
        times, gains = zip(*gains)
        profits = np.array(gains).cumsum()
        equity = pd.Series(profits, times)
        equity.plot(figsize=(14, 8), grid = True, label="secondary strategies")
        plt.axvline(x=OOS_date ,color="red")
def plot_equity_full(dna,secondary_equity):
    """
    Funzione che grafichi l'equity line risultatnte 
    dato un DNA.
    """
    gains = all_gains_full(dna)
    if len(gains)>0:
        gains=stack_sort(gains,secondary_equity)
        times, gains = zip(*gains)
        profits = np.array(gains).cumsum()
        equity = pd.Series(profits, times)
        equity.plot(figsize=(14, 8), grid = True, color="green")
        plt.axvline(x=OOS_date ,color="red")
  

In [ ]:
from numba import jit
@jit(nopython=True)
def numba_engine(rule,TIME_EXIT,target_monetario,COSTS_FIXED):
    
    for el in range(len(rule)):
        if rule[el] == 1:
            rule[el+1:el+TIME_EXIT+1]=0
    d=rule*target_monetario
    d[d != 0 ]=d[d != 0 ]-(COSTS_FIXED*2)
    return d

def target(DIRECTION,TIME_EXIT,dataset):
    price=dataset.open.copy()
    t_m=[]
    t_m=price.shift(-TIME_EXIT)-price
    t_m=t_m.shift(-1)
    t_m=t_m.fillna(0)
    if DIRECTION == "long":
        t_m=np.array(t_m * BIGPOINTVALUE)
    else:
        t_m=t_m*(-1)
        t_m=np.array(t_m * BIGPOINTVALUE)
    return t_m

def all_gains(dna):   
    rule = setups(numba.typed.List(dna))
    rule.setflags(write=1)
    rule=rule[:Out_Of_Sample_pct]
    gains = numba_engine(rule,TIME_EXIT,gains_array,COSTS_FIXED)
    tmp_times_array = times_array
    ls=[]
    for e in range(len(gains)):
        if gains[e]!=0:
            ls.append([tmp_times_array[e],gains[e]])
    return ls

def all_gains_full(dna):   
    rule = setups(numba.typed.List(dna))
    rule.setflags(write=1)
    gains = numba_engine(rule,TIME_EXIT,gains_array,COSTS_FIXED)
    tmp_times_array = times_array
    ls=[]
    for e in range(len(gains)):
        if gains[e]!=0:
            ls.append([tmp_times_array[e],gains[e]])
    return ls

def OOS(dataset,pct):
    if pct == 0:
        return len(dataset)
    else:
        return int(len(dataset) - (len(dataset) // (100/pct)))
    
def instability(array,pct):
    dimension = int(len(array) - (len(array) // (100/pct)))
    rng = np.random.default_rng()
    rand_array=rng.choice(array, dimension)
    return np.array(sorted(sorted(rand_array,key=lambda e:e[1]),key=lambda e:e[0]))

def check_secondary_equity(secondary_equity):
    if len(secondary_equity)>1:
        return secondary_equity
    else:
        return [np.datetime64(start_test+'T16:00:00.000000000'),0]    

In [ ]:
def pattern_translator():    
    
    

    pattern_dict = {}
    
    pattern_dict['p1'] = ['myBodyD1 = round(abs(dataset.daily_open - dataset.daily_close),2)',
                          'myRangeD1 = dataset.daily_high - dataset.daily_low',
                          'dataset["p1"] = myBodyD1<(0.1*myRangeD1)']
                          
    pattern_dict['p2'] = ['myBodyD1 = round(abs(dataset.daily_open - dataset.daily_close),2)',
                          'myRangeD1 = dataset.daily_high - dataset.daily_low',
                          'dataset["p2"] = myBodyD1<(0.25*myRangeD1)']  
    
    pattern_dict['p3'] = ['myBodyD1 = round(abs(dataset.daily_open - dataset.daily_close),2)',
                          'myRangeD1 = dataset.daily_high - dataset.daily_low',
                          'dataset["p3"]= myBodyD1<(0.5*myRangeD1)']
    
    pattern_dict['p4'] = ['myBodyD1 = round(abs(dataset.daily_open - dataset.daily_close),2)',
                          'myRangeD1 = dataset.daily_high - dataset.daily_low',
                          'dataset["p4"] = myBodyD1<(0.75*myRangeD1)'] 
    
    pattern_dict['p5'] = ['myBodyD1 = round(abs(dataset.daily_open - dataset.daily_close),2)',
                          'myRangeD1 = dataset.daily_high - dataset.daily_low',
                          'dataset["p5"] = myBodyD1>(0.25*myRangeD1)'] 

    pattern_dict['p6'] = ['myBodyD1 = round(abs(dataset.daily_open - dataset.daily_close),2)',
                          'myRangeD1 = dataset.daily_high - dataset.daily_low',
                          'dataset["p6"] = myBodyD1>(0.5*myRangeD1)'] 
    
    pattern_dict['p7'] = ['myBodyD1 = round(abs(dataset.daily_open - dataset.daily_close),2)',
                          'myRangeD1 = dataset.daily_high - dataset.daily_low',
                          'dataset["p7"]= myBodyD1>(0.75*myRangeD1)'] 
    
    pattern_dict['p8'] = ['myBodyD1 = round(abs(dataset.daily_open - dataset.daily_close),2)',
                          'myRangeD1 = dataset.daily_high - dataset.daily_low',
                          'dataset["p8"] = myBodyD1>(1*myRangeD1)'] 
    
    pattern_dict['p9'] = ['myBodyD1 = round(abs(dataset.daily_open - dataset.daily_close),2)',
                          'daily_high5 = daily_high(dataset,5)',
                          'daily_low5 = daily_low(dataset,5)',
                          'myRangeD5 = daily_high5 - daily_low5',
                          'dataset["p9"] = myBodyD1<(0.5*myRangeD5)']
    
    pattern_dict['p10'] = ['myBodyD1 = round(abs(dataset.daily_open - dataset.daily_close),2)',
                           'daily_high2 = daily_high(dataset,2)',
                           'daily_high3 = daily_high(dataset,3)',
                           'daily_high4 = daily_high(dataset,4)',
                           'daily_high5 = daily_high(dataset,5)',
                           'daily_high6 = daily_high(dataset,6)',
                           'daily_high7 = daily_high(dataset,7)',
                           'daily_high8 = daily_high(dataset,8)',
                           'daily_high9 = daily_high(dataset,9)',
                           'daily_high0 = daily_high(dataset,10)',
                           'daily_low2 = daily_low(dataset,2)',
                           'daily_low3 = daily_low(dataset,3)',
                           'daily_low4 = daily_low(dataset,4)',
                           'daily_low5 = daily_low(dataset,5)',
                           'daily_low6 = daily_low(dataset,5)',
                           'daily_low7 = daily_low(dataset,7)',
                           'daily_low8 = daily_low(dataset,8)',
                           'daily_low9 = daily_low(dataset,9)',
                           'daily_low0 = daily_low(dataset,10)',  
                           'myRangeMaxD10 = (highest_serie(dataset.daily_high,daily_high2,daily_high3,daily_high4,daily_high5,daily_high6,daily_high7,daily_high8,daily_high9,daily_high0) -\
                           (lowest_serie(dataset.daily_low,daily_low2,daily_low3,daily_low4,daily_low5,daily_low6,daily_low7,daily_low8,daily_low9,daily_low0)))',
                           'dataset["p10"] = myBodyD1<(0.5*myRangeMaxD10)']
    
    pattern_dict['p11'] = ['daily_high5 = daily_high(dataset,5)',
                          'daily_open5 = daily_open(dataset,5)',
                          'myBodyD5 = round(abs(daily_open5 - dataset.daily_close),2)',
                          'dataset["p11"] = myBodyD5 < (0.1*(daily_high5 - dataset.daily_low))']
    
    pattern_dict['p12'] = ['daily_high5 = daily_high(dataset,5)',
                          'daily_open5 = daily_open(dataset,5)',
                          'myBodyD5 = round(abs(daily_open5 - dataset.daily_close),2)',
                          'dataset["p12"] = myBodyD5 < (0.25*(daily_high5-dataset.daily_low))']
    
    pattern_dict['p13'] = ['daily_high5 = daily_high(dataset,5)',
                          'daily_open5 = daily_open(dataset,5)',
                          'myBodyD5 = round(abs(daily_open5 - dataset.daily_close),2)',
                          'dataset["p13"] = myBodyD5 < (0.50*(daily_high5-dataset.daily_low))']
    
    pattern_dict['p14'] = ['daily_high5 = daily_high(dataset,5)',
                          'daily_open5 = daily_open(dataset,5)',
                          'myBodyD5 = round(abs(daily_open5 - dataset.daily_close),2)',
                          'dataset["p14"] = myBodyD5 < (0.75*(daily_high5-dataset.daily_low))']
    
    pattern_dict['p15'] = ['daily_high5 = daily_high(dataset,5)',
                          'daily_open5 = daily_open(dataset,5)',
                          'myBodyD5 = round(abs(daily_open5 - dataset.daily_close),2)',
                          'dataset["p15"] = myBodyD5 < (1*(daily_high5-dataset.daily_low))']
    
    pattern_dict['p16'] = ['daily_high5 = daily_high(dataset,5)',
                          'daily_open5 = daily_open(dataset,5)',
                          'myBodyD5 = round(abs(daily_open5 - dataset.daily_close),2)',
                          'dataset["p16"] = myBodyD5 < (1.5*(daily_high5-dataset.daily_low))']
    
    pattern_dict['p17'] = ['daily_high5 = daily_high(dataset,5)',
                          'daily_open5 = daily_open(dataset,5)',
                          'myBodyD5 = round(abs(daily_open5 - dataset.daily_close),2)',
                          'dataset["p17"] = myBodyD5 < (2*(daily_high5-dataset.daily_low))']
    
    pattern_dict['p18'] = ['daily_high5 = daily_high(dataset,5)',
                          'daily_open5 = daily_open(dataset,5)',
                          'myBodyD5 = round(abs(daily_open5 - dataset.daily_close),2)',
                          'dataset["p18"] = myBodyD5 > (0.25*(daily_high5-dataset.daily_low))']
    
    pattern_dict['p19'] = ['daily_high5 = daily_high(dataset,5)',
                          'daily_open5 = daily_open(dataset,5)',
                          'myBodyD5 = round(abs(daily_open5 - dataset.daily_close),2)',
                          'dataset["p19"] = myBodyD5 > (0.50*(daily_high5-dataset.daily_low))']
    
    pattern_dict['p20'] = ['daily_high5 = daily_high(dataset,5)',
                          'daily_open5 = daily_open(dataset,5)',
                          'myBodyD5 = round(abs(daily_open5 - dataset.daily_close),2)',
                          'dataset["p20"] = myBodyD5 > (0.75*(daily_high5-dataset.daily_low))']
    
    pattern_dict['p21'] = ['daily_high5 = daily_high(dataset,5)',
                          'daily_open5 = daily_open(dataset,5)',
                          'myBodyD5 = round(abs(daily_open5 - dataset.daily_close),2)',
                          'dataset["p21"] = myBodyD5 > (1*(daily_high5-dataset.daily_low))']
    
    pattern_dict['p22'] = ['daily_high5 = daily_high(dataset,5)',
                          'daily_open5 = daily_open(dataset,5)',
                          'myBodyD5 = round(abs(daily_open5 - dataset.daily_close),2)',
                          'dataset["p22"] = myBodyD5 > (1.5*(daily_high5-dataset.daily_low))']
    
    pattern_dict['p23'] = ['daily_high5 = daily_high(dataset,5)',
                          'daily_open5 = daily_open(dataset,5)',
                          'myBodyD5 = round(abs(daily_open5 - dataset.daily_close),2)',
                          'dataset["p23"] = myBodyD5 > (2*(daily_high5-dataset.daily_low))']
    
    pattern_dict['p24'] = ['daily_high5 = daily_high(dataset,5)',
                          'daily_open5 = daily_open(dataset,5)',
                          'myBodyD5 = round(abs(daily_open5 - dataset.daily_close),2)',
                          'dataset["p24"] = myBodyD5 > (2.5*(daily_high5-dataset.daily_low))']
    
    pattern_dict['p25'] = ['daily_high5 = daily_high(dataset,5)',
                           'daily_low5 = daily_low(dataset,5)',
                           'daily_open5 = daily_open(dataset,5)',
                           'myBodyD5 = round(abs(daily_open5 - dataset.daily_close),2)',
                           'myRangeD5 = daily_high5 - daily_low5',                           
                           'dataset["p25"] = myBodyD5 < (0.1*myRangeD5)']
    
    pattern_dict['p26'] = ['daily_high5 = daily_high(dataset,5)',
                           'daily_low5 = daily_low(dataset,5)',
                           'daily_open5 = daily_open(dataset,5)',
                           'myBodyD5 = round(abs(daily_open5 - dataset.daily_close),2)',
                           'myRangeD5 = daily_high5 - daily_low5',                           
                           'dataset["p26"] = myBodyD5 < (0.25*myRangeD5)']
    
    pattern_dict['p27'] = ['daily_high5 = daily_high(dataset,5)',
                           'daily_low5 = daily_low(dataset,5)',
                           'daily_open5 = daily_open(dataset,5)',
                           'myBodyD5 = round(abs(daily_open5 - dataset.daily_close),2)',
                           'myRangeD5 = daily_high5 - daily_low5',                           
                           'dataset["p27"] = myBodyD5 < (0.5*myRangeD5)']
    
    pattern_dict['p28'] = ['daily_high5 = daily_high(dataset,5)',
                           'daily_low5 = daily_low(dataset,5)',
                           'daily_open5 = daily_open(dataset,5)',
                           'myBodyD5 = round(abs(daily_open5 - dataset.daily_close),2)',
                           'myRangeD5 = daily_high5 - daily_low5',                           
                           'dataset["p28"] = myBodyD5 < (0.75*myRangeD5)']
    
    pattern_dict['p29'] = ['daily_high5 = daily_high(dataset,5)',
                           'daily_low5 = daily_low(dataset,5)',
                           'daily_open5 = daily_open(dataset,5)',
                           'myBodyD5 = round(abs(daily_open5 - dataset.daily_close),2)',
                           'myRangeD5 = daily_high5 - daily_low5',                           
                           'dataset["p29"] = myBodyD5 > (0.1*myRangeD5)']
    
    pattern_dict['p30'] = ['daily_high5 = daily_high(dataset,5)',
                           'daily_low5 = daily_low(dataset,5)',
                           'daily_open5 = daily_open(dataset,5)',
                           'myBodyD5 = round(abs(daily_open5 - dataset.daily_close),2)',
                           'myRangeD5 = daily_high5 - daily_low5',                           
                           'dataset["p30"] = myBodyD5 > (0.25*myRangeD5)']
    
    pattern_dict['p31'] = ['daily_high5 = daily_high(dataset,5)',
                           'daily_low5 = daily_low(dataset,5)',
                           'daily_open5 = daily_open(dataset,5)',
                           'myBodyD5 = round(abs(daily_open5 - dataset.daily_close),2)',
                           'myRangeD5 = daily_high5 - daily_low5',                           
                           'dataset["p31"] = myBodyD5 > (0.5*myRangeD5)']
    
    pattern_dict['p32'] = ['daily_high5 = daily_high(dataset,5)',
                           'daily_low5 = daily_low(dataset,5)',
                           'daily_open5 = daily_open(dataset,5)',
                           'myBodyD5 = round(abs(daily_open5 - dataset.daily_close),2)',
                           'myRangeD5 = daily_high5 - daily_low5',                           
                           'dataset["p32"] = myBodyD5 > (0.75*myRangeD5)']
    
    pattern_dict['p33'] = ['daily_close2 = daily_close(dataset,2)',
                           'daily_close3 = daily_close(dataset,3)',
                           'daily_close4 = daily_close(dataset,4)',
                           'dataset["p33"] = (dataset.daily_close>daily_close2) & (daily_close2>daily_close3) & (daily_close3>daily_close4)']
    
    pattern_dict['p34'] = ['daily_close2 = daily_close(dataset,2)',
                           'daily_close3 = daily_close(dataset,3)',
                           'daily_close4 = daily_close(dataset,4)',
                           'dataset["p34"] = (dataset.daily_close<daily_close2) & (daily_close2<daily_close3) & (daily_close3<daily_close4)']
    
    pattern_dict['p35'] = ['daily_close2 = daily_close(dataset,2)',
                           'daily_close3 = daily_close(dataset,3)',
                           'daily_close4 = daily_close(dataset,4)',
                           'daily_close5 = daily_close(dataset,5)',
                           'dataset["p35"] = (dataset.daily_close>daily_close2) & (daily_close2>daily_close3) & (daily_close3>daily_close4) & (daily_close4>daily_close5)']
    
    pattern_dict['p36'] = ['daily_close2 = daily_close(dataset,2)',
                           'daily_close3 = daily_close(dataset,3)',
                           'daily_close4 = daily_close(dataset,4)',
                           'daily_close5 = daily_close(dataset,5)',
                           'dataset["p35"] = dataset["p36"] = (dataset.daily_close<daily_close2) & (daily_close2<daily_close3) & (daily_close3<daily_close4) & (daily_close4<daily_close5)']
    
    pattern_dict['p37'] = ['daily_high2 = daily_high(dataset,2)',
                           'daily_low2 = daily_low(dataset,2)',  
                           'dataset["p37"] = (dataset.daily_high>daily_high2) & (dataset.daily_low>daily_low2)']
    
    pattern_dict['p38'] = ['daily_high2 = daily_high(dataset,2)',
                           'daily_low2 = daily_low(dataset,2)',  
                           'dataset["p38"] = (dataset.daily_high<daily_high2) & (dataset.daily_low<daily_low2)']
    
    pattern_dict['p39'] = ['daily_close2 = daily_close(dataset,2)',                             
                           'dataset["p39"] = (dataset.daily_close>daily_close2)']
    
    pattern_dict['p40'] = ['daily_close2 = daily_close(dataset,2)',                             
                           'dataset["p40"] = (dataset.daily_close<daily_close2)']
    
    pattern_dict['p41'] = ['dataset["p41"] = (dataset.daily_close<dataset.daily_open)']
    
    pattern_dict['p42'] = ['dataset["p42"] = (dataset.daily_close>dataset.daily_open)']
    
    pattern_dict['p43'] = ['daily_close2 = daily_close(dataset,2)', 
                           'dataset["p43"] = (dataset.daily_close<(daily_close2-(daily_close2*0.5*0.01)))']
    
    pattern_dict['p44'] = ['daily_close2 = daily_close(dataset,2)', 
                           'dataset["p44"] = (dataset.daily_close<(daily_close2-(daily_close2*1*0.01)))']
    
    pattern_dict['p45'] = ['daily_close2 = daily_close(dataset,2)', 
                           'dataset["p45"] = (dataset.daily_close<(daily_close2-(daily_close2* 1.5*0.01)))']
    
    pattern_dict['p46'] = ['daily_close2 = daily_close(dataset,2)', 
                           'dataset["p46"] = (dataset.daily_close<(daily_close2-(daily_close2*2*0.01)))']
    
    pattern_dict['p47'] = ['daily_close2 = daily_close(dataset,2)', 
                           'dataset["p47"] = (dataset.daily_close<(daily_close2-(daily_close2*2.5*0.01)))']
    
    pattern_dict['p48'] = ['daily_close2 = daily_close(dataset,2)', 
                           'dataset["p48"] = (dataset.daily_close<(daily_close2-(daily_close2*3*0.01)))']
    
    pattern_dict['p49'] = ['daily_close2 = daily_close(dataset,2)', 
                           'dataset["p49"] = (dataset.daily_close>(daily_close2+(daily_close2*0.5*0.01)))']
    
    pattern_dict['p50'] = ['daily_close2 = daily_close(dataset,2)', 
                           'dataset["p50"] = (dataset.daily_close>(daily_close2+(daily_close2*1*0.01)))']
    
    pattern_dict['p51'] = ['daily_close2 = daily_close(dataset,2)', 
                           'dataset["p51"] = (dataset.daily_close>(daily_close2+(daily_close2*1.5*0.01)))']
    
    pattern_dict['p52'] = ['daily_close2 = daily_close(dataset,2)', 
                           'dataset["p52"] = (dataset.daily_close>(daily_close2+(daily_close2*2*0.01)))']
    
    pattern_dict['p53'] = ['daily_close2 = daily_close(dataset,2)', 
                           'dataset["p53"] = (dataset.daily_close>(daily_close2+(daily_close2*2.5*0.01)))']
    
    pattern_dict['p54'] = ['daily_close2 = daily_close(dataset,2)', 
                           'dataset["p54"] = (dataset.daily_close>(daily_close2+(daily_close2*3*0.01)))']
    
    pattern_dict['p55'] = ['daily_high5 = daily_high(dataset,5)', 
                           'dataset["p55"] = dataset.daily_high>daily_high5']
    
    pattern_dict['p56'] = ['daily_high5 = daily_high(dataset,5)', 
                           'dataset["p56"] = dataset.daily_high<daily_high5']
    
    pattern_dict['p57'] = ['daily_low5 = daily_low(dataset,5)', 
                           'dataset["p57"] = dataset.daily_low>daily_low5']
    
    pattern_dict['p58'] = ['daily_low5 = daily_low(dataset,5)', 
                           'dataset["p58"] = dataset.daily_low<daily_low5']
    
    pattern_dict['p59'] = ['daily_high2 = daily_high(dataset,2)',
                           'daily_high3 = daily_high(dataset,3)',
                           'daily_high4 = daily_high(dataset,4)',                           
                           'dataset["p59"] = (dataset.daily_high>daily_high2)&(dataset.daily_high>daily_high3)&(dataset.daily_high>daily_high4)']
    
    pattern_dict['p59'] = ['daily_high2 = daily_high(dataset,2)',
                           'daily_high3 = daily_high(dataset,3)',
                           'daily_high4 = daily_high(dataset,4)',                           
                           'dataset["p59"] = (dataset.daily_high>daily_high2)&(dataset.daily_high>daily_high3)&(dataset.daily_high>daily_high4)']
    
    pattern_dict['p60'] = ['daily_high2 = daily_high(dataset,2)',
                           'daily_high3 = daily_high(dataset,3)',
                           'daily_high4 = daily_high(dataset,4)',                           
                           'dataset["p60"] = (dataset.daily_high<daily_high2)&(dataset.daily_high<daily_high3)&(dataset.daily_high<daily_high4)']
    
    pattern_dict['p61'] = ['daily_low2 = daily_low(dataset,2)',
                           'daily_low3 = daily_low(dataset,3)',
                           'daily_low4 = daily_low(dataset,4)',                           
                           'dataset["p61"] = (dataset.daily_low<daily_low2)&(dataset.daily_low<daily_low3)&(dataset.daily_low<daily_low4)']
    
    pattern_dict['p62'] = ['daily_low2 = daily_low(dataset,2)',
                           'daily_low3 = daily_low(dataset,3)',
                           'daily_low4 = daily_low(dataset,4)',                           
                           'dataset["p62"] = (dataset.daily_low>daily_low2)&(dataset.daily_low>daily_low3)&(dataset.daily_low>daily_low4)']
    
    pattern_dict['p63'] = ['daily_close2 = daily_close(dataset,2)',
                           'daily_close3 = daily_close(dataset,3)',                                                
                           'dataset["p63"] = (dataset.daily_close>daily_close2)&(dataset.daily_close>daily_close3)&(dataset.daily_close>dataset.daily_open)']
    
    pattern_dict['p64'] = ['daily_close2 = daily_close(dataset,2)',
                           'daily_close3 = daily_close(dataset,3)',                                                
                           'dataset["p64"] = (dataset.daily_close<daily_close2)&(dataset.daily_close<daily_close3)&(dataset.daily_close<dataset.daily_open)']
    
    pattern_dict['p65'] = ['myRangeD1 = dataset.daily_high - dataset.daily_low',                                              
                           'dataset["p65"] = (dataset.daily_high-dataset.daily_close)<0.20*myRangeD1']
    
    pattern_dict['p66'] = ['myRangeD1 = dataset.daily_high - dataset.daily_low',                                              
                           'dataset["p66"] = (dataset.daily_high-dataset.daily_low)<0.20*myRangeD1']
    
    pattern_dict['p67'] = ['myRangeD1 = dataset.daily_high - dataset.daily_low',
                           'daily_high2 = daily_high(dataset,2)',
                           'daily_low2 = daily_low(dataset,2)',                           
                           'dataset["p67"] = (myRangeD1<(((daily_high2-daily_low2)+(daily_high2-daily_low2))/3))']
    
    pattern_dict['p68'] = ['myRangeD1 = dataset.daily_high - dataset.daily_low',
                           'daily_high2 = daily_high(dataset,2)',
                           'daily_high3 = daily_high(dataset,3)',
                           'daily_low2 = daily_low(dataset,2)',
                           'daily_low3 = daily_low(dataset,3)',
                           'myRangeD2 = daily_high2 - daily_low2',
                           'myRangeD3 = daily_high3 - daily_low3',
                           'dataset["p68"] = (myRangeD1<myRangeD2)&(myRangeD2<myRangeD3)']
    
    pattern_dict['p69'] = ['daily_high2 = daily_high(dataset,2)',
                           'daily_low2 = daily_low(dataset,2)',                           
                           'dataset["p69"] = (daily_high2>dataset.daily_high)&(daily_low2 < dataset.daily_low)']
    
    pattern_dict['p70'] = ['daily_high2 = daily_high(dataset,2)',
                           'dataset["p70"] = (dataset.daily_high<daily_high2)']
    
    pattern_dict['p71'] = ['daily_low2 = daily_low(dataset,2)',
                           'dataset["p71"] = (dataset.daily_low>daily_low2)']
    
    pattern_dict['p72'] = ['daily_low2 = daily_low(dataset,2)',
                           'dataset["p72"] = (dataset.daily_low<daily_low2)']
    
    pattern_dict['p73'] = ['daily_high2 = daily_high(dataset,2)',
                           'daily_low2 = daily_low(dataset,2)',                            
                           'dataset["p73"] = (daily_high2>dataset.daily_high)|(daily_low2<dataset.daily_low)']
    
    pattern_dict['p74'] = ['daily_high2 = daily_high(dataset,2)',
                           'daily_low2 = daily_low(dataset,2)',                            
                           'dataset["p74"] = (daily_high2<dataset.daily_high)&(daily_low2>dataset.daily_low)']
    
    pattern_dict['p75'] = ['daily_close2 = daily_close(dataset,2)',
                           'daily_open2 = daily_open(dataset,2)',                            
                           'dataset["p75"] = (dataset.daily_close>dataset.daily_open) & (daily_close2>daily_open2)']
    
    pattern_dict['p76'] = ['daily_close2 = daily_close(dataset,2)',
                           'daily_open2 = daily_open(dataset,2)',                            
                           'dataset["p76"] = (dataset.daily_close<dataset.daily_open) & (daily_close2>daily_open2)']
    
    pattern_dict['p77'] = ['daily_close2 = daily_close(dataset,2)',
                           'daily_open2 = daily_open(dataset,2)',                            
                           'dataset["p77"] = (dataset.daily_close<dataset.daily_open) & (daily_close2<daily_open2)']
    
    pattern_dict['p78'] = ['daily_open2 = daily_open(dataset,2)',
                           'daily_high2 = daily_high(dataset,2)',
                           'daily_low2 = daily_low(dataset,2)',                           
                           'myMaxHighD2 = highest_serie(dataset.daily_high,daily_high2)',
                           'myMinLowD2 = lowest_serie(dataset.daily_low,daily_low2)',  
                           'dataset["p78"] = abs(daily_open2-dataset.daily_close) < (0.5*(myMaxHighD2 - myMinLowD2))']
    
    pattern_dict['p79'] = ['daily_high2 = daily_high(dataset,2)',                          
                           'dataset["p79"] = (dataset.daily_high> daily_high2)']
    
    pattern_dict['p80'] = ['daily_high2 = daily_high(dataset,2)',   
                           'daily_high3 = daily_high(dataset,3)',
                           'dataset["p80"] = (dataset.daily_high> daily_high2) & (dataset.daily_high>daily_high3)']
    
    pattern_dict['p81'] = ['daily_high2 = daily_high(dataset,2)',   
                           'daily_high3 = daily_high(dataset,3)',
                           'dataset["p81"] = (dataset.daily_high< daily_high2) & (dataset.daily_high< daily_high3)']
    
    pattern_dict['p82'] = ['daily_low2 = daily_low(dataset,2)',                          
                           'dataset["p82"] = (dataset.daily_low<daily_low2)']
    
    pattern_dict['p82'] = ['daily_low2 = daily_low(dataset,2)', 
                           'daily_low3 = daily_low(dataset,2)',                            
                           'dataset["p82"] = (dataset.daily_low>daily_low2) & (dataset.daily_low>daily_low3)']
    
    pattern_dict['p83'] = ['daily_low2 = daily_low(dataset,2)', 
                           'daily_low3 = daily_low(dataset,2)',                            
                           'dataset["p83"] = (dataset.daily_low<daily_low2) & (dataset.daily_low<daily_low3)']
    
    pattern_dict['p84'] = ['daily_low2 = daily_low(dataset,2)', 
                           'daily_low3 = daily_low(dataset,2)', 
                           'daily_high2 = daily_high(dataset,2)', 
                           'daily_high3 = daily_high(dataset,3)', 
                           'dataset["p84"] = (dataset.daily_high>daily_high2) & (dataset.daily_low>daily_low2) & (daily_high2<daily_high3) & (daily_low2<daily_low3)']
    
    pattern_dict['p85'] = ['daily_low2 = daily_low(dataset,2)', 
                           'daily_low3 = daily_low(dataset,2)', 
                           'daily_high2 = daily_high(dataset,2)', 
                           'daily_high3 = daily_high(dataset,3)', 
                           'dataset["p85"] = (dataset.daily_high<daily_high2) & (dataset.daily_low<daily_low2) & (daily_high2>daily_high3) & (daily_low2>daily_low3)']


    pattern_dict['p86'] = ['daily_close2 = daily_close(dataset,2)',
                           'daily_close3 = daily_close(dataset,3)',
                           'dataset["p86"] = (dataset.daily_close>daily_close2) & (daily_close2>daily_close3)']
    
    pattern_dict['p87'] = ['daily_close2 = daily_close(dataset,2)',
                           'daily_close3 = daily_close(dataset,3)',
                           'dataset["p87"] = (dataset.daily_close<daily_close2) & (daily_close2<daily_close3)']
    
    pattern_dict['p88'] = ['daily_open2 = daily_open(dataset,2)',
                           'daily_high2 = daily_high(dataset,2)',
                           'daily_low2 = daily_low(dataset,2)',
                           'daily_close2 = daily_close(dataset,2)',                           
                           'myAvgPriceD1 = round(((dataset.daily_open + dataset.daily_high +dataset.daily_low + dataset.daily_close)/4),2)',
                           'myAvgPriceD2 = round(((daily_open2 + daily_high2 + daily_low2 + daily_close2)/4),2)',
                           'dataset["p88"] = (myAvgPriceD1 > myAvgPriceD2)']
    
    pattern_dict['p89'] = ['daily_open2 = daily_open(dataset,2)',
                           'daily_high2 = daily_high(dataset,2)',
                           'daily_low2 = daily_low(dataset,2)',
                           'daily_close2 = daily_close(dataset,3)', 
                           'daily_open3 = daily_open(dataset,3)',
                           'daily_high3 = daily_high(dataset,3)',
                           'daily_low3 = daily_low(dataset,3)',
                           'daily_close3 = daily_close(dataset,3)', 
                           'myAvgPriceD1 = round(((dataset.daily_open + dataset.daily_high + dataset.daily_low + dataset.daily_close)/4),2)',
                           'myAvgPriceD2 = round(((daily_open2 + daily_high2 + daily_low2 + daily_close2)/4),2)',
                           'myAvgPriceD3 = round(((daily_open3 + daily_high3 + daily_low3 + daily_close3)/4),2)',
                           'dataset["p89"] = (myAvgPriceD1 > myAvgPriceD2) & (myAvgPriceD1 > myAvgPriceD3)']
    
    pattern_dict['p90'] = ['daily_open2 = daily_open(dataset,2)',
                           'daily_high2 = daily_high(dataset,2)',
                           'daily_low2 = daily_low(dataset,2)',
                           'daily_close2 = daily_close(dataset,2)',                           
                           'myAvgPriceD1 = round(((dataset.daily_open + dataset.daily_high + dataset.daily_low + dataset.daily_close)/4),2)',
                           'myAvgPriceD2 = round(((daily_open2 + daily_high2 + daily_low2 + daily_close2)/4),2)',
                           'dataset["p90"] = (myAvgPriceD1 < myAvgPriceD2)']
    
    pattern_dict['p91'] = ['daily_open2 = daily_open(dataset,2)',
                           'daily_high2 = daily_high(dataset,2)',
                           'daily_low2 = daily_low(dataset,2)',
                           'daily_close2 = daily_close(dataset,3)', 
                           'daily_open3 = daily_open(dataset,3)',
                           'daily_high3 = daily_high(dataset,3)',
                           'daily_low3 = daily_low(dataset,3)',
                           'daily_close3 = daily_close(dataset,3)', 
                           'myAvgPriceD1 = round(((dataset.daily_open + dataset.daily_high + dataset.daily_low + dataset.daily_close)/4),2)',
                           'myAvgPriceD2 = round(((daily_open2 + daily_high2 + daily_low2 + daily_close2)/4),2)',
                           'myAvgPriceD3 = round(((daily_open3 + daily_high3 + daily_low3 + daily_close3)/4),2)',
                           'dataset["p91"] = (myAvgPriceD1 < myAvgPriceD2) & (myAvgPriceD1 < myAvgPriceD3)']

    pattern_dict['p92'] = ['daily_high2 = daily_high(dataset,2)',
                           'dataset["p92"] = (dataset.daily_close>daily_high2)']
    
    pattern_dict['p93'] = ['daily_high2 = daily_high(dataset,2)',
                           'dataset["p93"] = (dataset.daily_close<daily_high2)']
    
    pattern_dict['p94'] = ['daily_high2 = daily_high(dataset,2)',
                           'daily_high3 = daily_high(dataset,3)',
                           'dataset["p94"] = (dataset.daily_close>daily_high2) & (dataset.daily_close>daily_high3)']
    
    pattern_dict['p95'] = ['daily_low2 = daily_low(dataset,2)',
                           'daily_low3 = daily_low(dataset,3)',
                           'dataset["p95"] = (dataset.daily_close<daily_low2) & (dataset.daily_close<daily_low3) ']
    
    pattern_dict['p96'] = ['dataset["p96"] = True']
    
    pattern_dict['p97'] = ['daily_low2 = daily_low(dataset,2)',
                           'daily_low3 = daily_low(dataset,3)',
                           'daily_low4 = daily_low(dataset,4)',
                           'dataset["p97"] = (dataset.daily_close<daily_low2) & (dataset.daily_close<daily_low3) & (dataset.daily_close<daily_low4)']
    
    pattern_dict['p98'] = ['daily_high2 = daily_high(dataset,2)',
                           'daily_open2 = daily_open(dataset,2)',
                           'dataset["p98"] = (dataset.daily_close>daily_high2) & (dataset.daily_close>dataset.daily_open) & (dataset.daily_close<daily_open2)']
    
    pattern_dict['p99'] = ['daily_low2 = daily_low(dataset,2)',
                           'daily_open2 = daily_open(dataset,2)',
                           'dataset["p99"] = (dataset.daily_close<daily_low2) & (dataset.daily_close<dataset.daily_open) & (dataset.daily_close>daily_open2)']
    
    pattern_dict['p100'] = ['daily_low2 = daily_low(dataset,2)',
                            'daily_high2 = daily_high(dataset,2)',
                            'myRangeD1 = dataset.daily_high - dataset.daily_low',
                            'myRangeD2 = daily_high2 - daily_low2',
                            'dataset["p100"] = (myRangeD1<myRangeD2)']
    
    pattern_dict['p101'] = ['daily_low5 = daily_low(dataset,5)',
                            'daily_high5 = daily_high(dataset,5)',
                            'myRangeD1 = dataset.daily_high - dataset.daily_low',
                            'myRangeD5 = daily_high5 - daily_low5',
                            'dataset["p101"] = (myRangeD1<myRangeD5)']
    
    pattern_dict['p102'] = ['daily_high2 = daily_high(dataset,2)',
                           'daily_high3 = daily_high(dataset,3)',
                           'daily_high4 = daily_high(dataset,4)',
                           'daily_high5 = daily_high(dataset,5)',
                           'daily_high6 = daily_high(dataset,6)',
                           'daily_high7 = daily_high(dataset,7)',
                           'daily_high8 = daily_high(dataset,8)',
                           'daily_high9 = daily_high(dataset,9)',
                           'daily_high10 = daily_high(dataset,10)',
                           'daily_low2 = daily_low(dataset,2)',
                           'daily_low3 = daily_low(dataset,3)',
                           'daily_low4 = daily_low(dataset,4)',
                           'daily_low5 = daily_low(dataset,5)',
                           'daily_low6 = daily_low(dataset,5)',
                           'daily_low7 = daily_low(dataset,7)',
                           'daily_low8 = daily_low(dataset,8)',
                           'daily_low9 = daily_low(dataset,9)',
                           'daily_low10 = daily_low(dataset,10)',  
                           'myRangeMaxD10 = (highest_serie(dataset.daily_high,daily_high2,daily_high3,daily_high4,daily_high5,daily_high6,daily_high7,daily_high8,daily_high9,daily_high10) - (lowest_serie(dataset.daily_low,daily_low2,daily_low3,daily_low4,daily_low5,daily_low6,daily_low7,daily_low8,daily_low9,daily_low10)))',
                           'myRangeD1 = dataset.daily_high - dataset.daily_low',                            
                           'dataset["p102"] = (myRangeD1<myRangeMaxD10)']
    
    pattern_dict['p103'] = ['daily_high2 = daily_high(dataset,2)',
                           'daily_high3 = daily_high(dataset,3)',
                           'daily_high4 = daily_high(dataset,4)',
                           'daily_high5 = daily_high(dataset,5)',
                           'daily_high6 = daily_high(dataset,6)',
                           'daily_high7 = daily_high(dataset,7)',
                           'daily_high8 = daily_high(dataset,8)',
                           'daily_high9 = daily_high(dataset,9)',
                           'daily_high0 = daily_high(dataset,10)',
                           'daily_low2 = daily_low(dataset,2)',
                           'daily_low3 = daily_low(dataset,3)',
                           'daily_low4 = daily_low(dataset,4)',
                           'daily_low5 = daily_low(dataset,5)',
                           'daily_low6 = daily_low(dataset,5)',
                           'daily_low7 = daily_low(dataset,7)',
                           'daily_low8 = daily_low(dataset,8)',
                           'daily_low9 = daily_low(dataset,9)',
                           'daily_low0 = daily_low(dataset,10)', 
                           'myRangeDelay1MaxD10 = highest_serie(daily_high2,daily_high3,daily_high4,daily_high5,daily_high6,daily_high7,daily_high8,daily_high9,daily_high0) - \
                            lowest_serie(daily_low2,daily_low3,daily_low4,daily_low5,daily_low6,daily_low7,daily_low8,daily_low9,daily_low0)',
                            'myRangeD1 = dataset.daily_high - dataset.daily_low',                            
                            'dataset["p103"] = (myRangeD1>myRangeDelay1MaxD10)']
    

    pattern_dict['p104'] = ['daily_high2 = daily_high(dataset,2)',
                            'daily_high3 = daily_high(dataset,3)',
                            'daily_high4 = daily_high(dataset,4)',
                            'myMaxDelay1HighD4 =  highest_serie(daily_high2,daily_high3,daily_high4)',
                            'dataset["p104"] = (dataset.daily_close>myMaxDelay1HighD4)']
    
    pattern_dict['p105'] = ['daily_high2 = daily_high(dataset,2)',
                            'daily_high3 = daily_high(dataset,3)',
                            'daily_high4 = daily_high(dataset,4)',
                            'daily_high5 = daily_high(dataset,5)',
                            'myMaxDelay1HighD5 = highest_serie(daily_high2,daily_high3,daily_high4,daily_high5)',
                            'dataset["p105"] = (dataset.daily_close>myMaxDelay1HighD5)']
    
    pattern_dict['p106'] = ['daily_low2 = daily_low(dataset,2)',
                           'daily_low3 = daily_low(dataset,3)',
                           'daily_low4 = daily_low(dataset,4)',
                            'myMinDelay1LowD4 = lowest_serie(daily_low2,daily_low3,daily_low4)',
                            'dataset["p106"] = (dataset.daily_close<myMinDelay1LowD4)']
    
    pattern_dict['p107'] = ['daily_low2 = daily_low(dataset,2)',
                           'daily_low3 = daily_low(dataset,3)',
                           'daily_low4 = daily_low(dataset,4)',
                           'daily_low5 = daily_low(dataset,5)',                           
                           'myMinDelay1LowD5 = lowest_serie(daily_low2,daily_low3,daily_low4,daily_low5)',
                           'dataset["p107"] = (dataset.daily_close<myMinDelay1LowD5)']
    
    pattern_dict['p108'] = ['dataset["p108"] = True ']
    pattern_dict['p109'] = ['dataset["p109"] = True ']
    pattern_dict['p110'] = ['dataset["p110"] = True ']
    pattern_dict['p111'] = ['dataset["p111"] = True ']
    pattern_dict['p112'] = ['dataset["p112"] = True ']
    pattern_dict['p113'] = ['dataset["p113"] = True ']

    
    pattern_dict['p114'] = ['daily_close2 = daily_close(dataset,2)',
                            'dataset["p114"] = (dataset.daily_open<daily_close2)']
    
    pattern_dict['p115'] = ['daily_low3 = daily_low(dataset,3)',
                            'dataset["p115"] = (dataset.daily_low<daily_low3)']
    
    pattern_dict['p116'] = ['daily_low2 = daily_low(dataset,2)',
                            'daily_high2 = daily_high(dataset,2)',
                            'dataset["p116"] = (dataset.daily_high-dataset.daily_low)<(daily_high2-daily_low2)']
    
    pattern_dict['p117'] = ['daily_high3 = daily_high(dataset,3)',
                            'dataset["p117"] = (dataset.daily_high>daily_high3)']
    
    pattern_dict['p118'] = ['daily_low2 = daily_low(dataset,2)',
                            'dataset["p118"] = (dataset.daily_low<daily_low2)']
    
    pattern_dict['p119'] = ['daily_high2 = daily_high(dataset,2)',
                            'dataset["p119"] = (dataset.daily_high>daily_high2)']
    
    pattern_dict['p120'] = ['daily_open5 = daily_open(dataset,5)',
                            'daily_high2 = daily_high(dataset,2)',
                            'daily_high3 = daily_high(dataset,3)',
                            'daily_high4 = daily_high(dataset,4)',
                            'daily_high5 = daily_high(dataset,5)',
                            'daily_low2 = daily_low(dataset,2)',
                            'daily_low3 = daily_low(dataset,3)',
                            'daily_low4 = daily_low(dataset,4)',
                            'daily_low5 = daily_low(dataset,5)',                            
                            'myMaxHighD5 = highest_serie(dataset.daily_high,daily_high2,daily_high3,daily_high4,daily_high5)',
                            'myMinLowD5 = lowest_serie(dataset.daily_low,daily_low2,daily_low3,daily_low4,daily_low5)',
                            'dataset["p120"] = (abs(daily_open5-dataset.daily_close)<0.66*(myMaxHighD5-myMinLowD5))']
    
    pattern_dict['p121'] = ['daily_high2 = daily_high(dataset,2)',
                            'daily_high3 = daily_high(dataset,3)',
                            'daily_high4 = daily_high(dataset,4)',
                            'daily_high5 = daily_high(dataset,5)',
                            'daily_low2 = daily_low(dataset,2)',
                            'daily_low3 = daily_low(dataset,3)',
                            'daily_low4 = daily_low(dataset,4)',
                            'daily_low5 = daily_low(dataset,5)',                            
                            'myMaxHighD5 = highest_serie(dataset.daily_high,daily_high2,daily_high3,daily_high4,daily_high5)',
                            'myMinLowD5 = lowest_serie(dataset.daily_low,daily_low2,daily_low3,daily_low4,daily_low5)',
                            'dataset["p121"] = (abs(dataset.daily_open-dataset.daily_close)<0.66*(myMaxHighD5-myMinLowD5))']
    
    
    pattern_dict['p122'] = ['dataset["p122"] = (abs(dataset.daily_open-dataset.daily_close)<0.66*(dataset.daily_high-dataset.daily_low))']
        
    pattern_dict['p123'] = ['daily_low2 = daily_low(dataset,2)',
                            'daily_high2 = daily_high(dataset,2)',
                            'daily_high3 = daily_high(dataset,3)',
                            'daily_low3 = daily_low(dataset,3)',
                            'daily_high4 = daily_high(dataset,4)',
                            'daily_low4 = daily_low(dataset,4)',                            
                            'dataset["p123"] = (dataset.daily_high-dataset.daily_low)>0.66*(((dataset.daily_high-dataset.daily_low)+\
                            (daily_high2-daily_low2)+(daily_high3-daily_low3)+\
                            (daily_high4-daily_low4)/4))']
    
    pattern_dict['p124'] = ['daily_low2 = daily_low(dataset,2)',
                            'daily_high2 = daily_high(dataset,2)',
                            'daily_high3 = daily_high(dataset,3)',
                            'daily_low3 = daily_low(dataset,3)',
                            'daily_high4 = daily_high(dataset,4)',
                            'daily_low4 = daily_low(dataset,4)',                            
                            'dataset["p124"] = (dataset.daily_high-dataset.daily_low)<0.66*(((dataset.daily_high-dataset.daily_low)+\
                            (daily_high2-daily_low2)+(daily_high3-daily_low3)+\
                            (daily_high4-daily_low4)/4))']
        
    
    pattern_dict['p125'] = ['weekly_low1 = weekly_low(dataset,1)',
                            'weekly_open1 = weekly_open(dataset,1)',
                            'weekly_close1 = weekly_close(dataset,1)',
                            'weekly_high1 = weekly_high(dataset,1)',        
                            'dataset["p125"] = abs((weekly_open1-weekly_close1)<0.75*(weekly_high1-weekly_low1))']
    
    pattern_dict['p125'] = ['weekly_low1 = weekly_low(dataset,1)',
                            'weekly_open1 = weekly_open(dataset,1)',
                            'weekly_close1 = weekly_close(dataset,1)',
                            'weekly_high1 = weekly_high(dataset,1)',        
                            'dataset["p125"] = abs((weekly_open1-weekly_close1)<0.75*(weekly_high1-weekly_low1))']
        
    pattern_dict['p126'] = ['weekly_low1 = weekly_low(dataset,1)',
                            'weekly_open1 = weekly_open(dataset,1)',
                            'weekly_close1 = weekly_close(dataset,1)',
                            'weekly_high1 = weekly_high(dataset,1)',        
                            'dataset["p126"] = abs((weekly_open1-weekly_close1)<0.50*(weekly_high1-weekly_low1))']
    
    pattern_dict['p127'] = ['weekly_low1 = weekly_low(dataset,1)',
                            'weekly_open1 = weekly_open(dataset,1)',
                            'weekly_close1 = weekly_close(dataset,1)',
                            'weekly_high1 = weekly_high(dataset,1)',        
                            'dataset["p127"] = abs((weekly_open1-weekly_close1)<0.25*(weekly_high1-weekly_low1))']
    
    pattern_dict['p128'] = ['daily_high2 = daily_high(dataset,2)',
                            'daily_high3 = daily_high(dataset,3)',
                            'daily_high4 = daily_high(dataset,4)',
                            'daily_low2 = daily_low(dataset,2)',
                            'daily_low3 = daily_low(dataset,3)',
                            'daily_low4 = daily_low(dataset,4)',       
                            'dataset["p128"] = ((daily_high4-daily_low4)<(daily_high3-daily_low3)) & ((daily_high3-daily_low3)<(daily_high2-daily_low2)) & ((daily_high2-daily_low2)<(dataset.daily_high-dataset.daily_low))']
    
    pattern_dict['p129'] = ['daily_high2 = daily_high(dataset,2)',
                            'daily_high3 = daily_high(dataset,3)',
                            'daily_high4 = daily_high(dataset,4)',
                            'daily_low2 = daily_low(dataset,2)',
                            'daily_low3 = daily_low(dataset,3)',
                            'daily_low4 = daily_low(dataset,4)',       
                            'dataset["p129"] = ((daily_high3-daily_low3)<(daily_high2-daily_low2))&((daily_high2-daily_low2)<(dataset.daily_high-dataset.daily_low))']
    
    pattern_dict['p130'] = ['daily_high2 = daily_high(dataset,2)',
                            'daily_high3 = daily_high(dataset,3)',
                            'daily_high4 = daily_high(dataset,4)',
                            'daily_low2 = daily_low(dataset,2)',
                            'daily_low3 = daily_low(dataset,3)',
                            'daily_low4 = daily_low(dataset,4)',       
                            'dataset["p130"] = ((daily_high4-daily_low4)>(daily_high3-daily_low3)) & ((daily_high3-daily_low3)>(daily_high2-daily_low2)) & ((daily_high2-daily_low2)>(dataset.daily_high-dataset.daily_low))']
    
    pattern_dict['p131'] = ['daily_high2 = daily_high(dataset,2)',
                            'daily_high3 = daily_high(dataset,3)',
                            'daily_high4 = daily_high(dataset,4)',
                            'daily_low2 = daily_low(dataset,2)',
                            'daily_low3 = daily_low(dataset,3)',
                            'daily_low4 = daily_low(dataset,4)',       
                            'dataset["p131"] = ((daily_high3-daily_low3)>(daily_high2-daily_low2)) & ((daily_high2-daily_low2)>(dataset.daily_high-dataset.daily_low))']
    
    pattern_dict['p132'] = ['dataset["p132"] = (dataset.daily_close-dataset.daily_low)<(dataset.daily_high-dataset.daily_low)/4']
    
    pattern_dict['p133'] = ['dataset["p133"] = (dataset.daily_high-dataset.daily_close)<(dataset.daily_high-dataset.daily_low)/4']
    
    pattern_dict['p134'] = ['dataset["p134"] = (dataset.daily_close-dataset.daily_low)<(dataset.daily_high-dataset.daily_low)/3']
    
    pattern_dict['p135'] = ['dataset["p135"] = (dataset.daily_high-dataset.daily_close)<(dataset.daily_high-dataset.daily_low)/4']
    
    pattern_dict['p136'] = ['dataset["p136"] = (dataset.daily_close-dataset.daily_low)<(dataset.daily_high-dataset.daily_low)/2']
    
    pattern_dict['p137'] = ['dataset["p137"] = (dataset.daily_high-dataset.daily_close)<(dataset.daily_high-dataset.daily_low)/2']
    
    pattern_dict['p138'] = ['daily_open0 = daily_open(dataset,0)',                            
                            'dataset["p138"] = (daily_open0 > dataset.daily_high)']
    
    pattern_dict['p139'] = ['daily_open0 = daily_open(dataset,0)',                            
                            'dataset["p139"] = (daily_open0<dataset.daily_high)']
    
    pattern_dict['p140'] = ['daily_open0 = daily_open(dataset,0)',                            
                            'dataset["p140"] = (daily_open0<dataset.daily_low)']
    
    pattern_dict['p141'] = ['daily_open0 = daily_open(dataset,0)',
                            'daily_high2 = daily_high(dataset,2)',                            
                            'dataset["p141"] = (daily_open0<=daily_high2)']
    
    pattern_dict['p142'] = ['daily_open0 = daily_open(dataset,0)',
                            'daily_high2 = daily_high(dataset,2)',                            
                            'dataset["p142"] = (daily_open0>daily_high2)']
    
    pattern_dict['p143'] = ['daily_open0 = daily_open(dataset,0)',
                            'daily_low3 = daily_low(dataset,3)',                            
                            'dataset["p143"] = (daily_open0>daily_low3)']
    
    pattern_dict['p144'] = ['daily_open0 = daily_open(dataset,0)',
                            'daily_open2 = daily_open(dataset,2)',                            
                            'dataset["p144"] = (daily_open0>daily_open2)']
    
    pattern_dict['p145'] = ['daily_open0 = daily_open(dataset,0)',
                            'daily_open2 = daily_open(dataset,2)',                            
                            'dataset["p145"] = (daily_open0<daily_open2)']
    
    pattern_dict['p146'] = ['daily_low2 = daily_low(dataset,2)',                            
                            'dataset["p146"] = (dataset.daily_close>daily_low2)']
    
    pattern_dict['p147'] = ['daily_low2 = daily_low(dataset,2)',                            
                            'dataset["p147"] = (dataset.daily_close<daily_low2)']
    
    pattern_dict['p148'] = ['daily_close2 = daily_close(dataset,2)',                            
                            'dataset["p148"] = (dataset.daily_close>daily_close2)']
    
    pattern_dict['p149'] = ['daily_close2 = daily_close(dataset,2)',                            
                            'dataset["p149"] = (dataset.daily_close<daily_close2)']
    
    pattern_dict['p150'] = ['daily_close2 = daily_close(dataset,2)',
                            'daily_close3 = daily_close(dataset,3)',                            
                            'dataset["p150"] = (daily_close2 < daily_close3)']

    return pattern_dict
